In [1]:
import requests
import pandas as pd
import time
from openpyxl import load_workbook

In [2]:
# Fetch cryptocurrency data
def fetch_crypto_data():
    url="https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency":"usd",
        "order":"market_cap_desc",
        "per_page":50,
        "page":1,
        "sparkline":False
    }
    response=requests.get(url, params=params)
    if response.status_code==200:
        return response.json()
    else:
        print("Error fetching data:", response.status_code)
        return None

In [3]:
# Analyze data
def analyze_data(data):
    df=pd.DataFrame(data)
    # Select relevant columns
    df=df[['name', 'symbol', 'current_price', 'market_cap', 'total_volume', 'price_change_percentage_24h']]
    # Top 5 cryptos by market cap
    top_5=df.nlargest(5, 'market_cap')
    # Average price of top 50
    avg_price=df['current_price'].mean()
    # Highest & lowest 24h percentage change
    highest_24h=df.nlargest(1, 'price_change_percentage_24h')
    lowest_24h=df.nsmallest(1, 'price_change_percentage_24h')
    print("\n🔹 **Analysis Report** 🔹")
    print(f"Top 5 Cryptos by Market Cap:\n{top_5[['name', 'market_cap']]}")
    print(f"\nAverage Price of Top 50: ${avg_price:.2f}")
    print(f"\n🔺 Highest 24h Change: {highest_24h['name'].values[0]} ({highest_24h['price_change_percentage_24h'].values[0]:.2f}%)")
    print(f"🔻 Lowest 24h Change: {lowest_24h['name'].values[0]} ({lowest_24h['price_change_percentage_24h'].values[0]:.2f}%)")
    
    return df

In [4]:
# Save data to an Excel sheet
def save_to_excel(df):
    file_name="Live_Crypto_Data.xlsx"
    try:
        # Load existing file if it exists
        book=load_workbook(file_name)
        writer=pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='replace')
        writer.book = book
    except FileNotFoundError:
        writer=pd.ExcelWriter(file_name, engine='openpyxl')
    df.to_excel(writer, index=False, sheet_name="Live Data")
    writer.close()
    print("✅ Excel file updated!")

In [5]:
# Continuous updating every 5 minutes
def live_update(interval=300):
    while True:
        data=fetch_crypto_data()
        if data:
            df=analyze_data(data)
            save_to_excel(df)
        print(f"⏳ Waiting {interval // 60} minutes before the next update...\n")
        time.sleep(interval)

In [ ]:
# Run live update every 5 minutes
live_update()


🔹 **Analysis Report** 🔹
Top 5 Cryptos by Market Cap:
       name     market_cap
0   Bitcoin  1946533142159
1  Ethereum   326624880703
2       XRP   144396995115
3    Tether   141901224912
4    Solana    99869251932

Average Price of Top 50: $4258.62

🔺 Highest 24h Change: Cardano (15.50%)
🔻 Lowest 24h Change: MANTRA (-2.87%)
✅ Excel file updated!
⏳ Waiting 5 minutes before the next update...

